In [1]:
import numpy as np
import tensorflow as tf
import scipy.io as sio
import os

In [2]:
data_path = os.path.join('E:\Eric_HSI\hyperspectral_datasets')
data = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))
labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))

window_size=19
input_size=200

In [3]:
features = data['data']
print(features.shape)
# 标签
labels = labels['groundT']
labels = np.reshape(labels, (-1, 1))   # 把标签变为2维， (21025, 1)
labels.shape

(145, 145, 200)


(21025, 1)

In [4]:
# 创建一个 (21025, 19, 19, 200) 的数组装数据
dataset = np.zeros((labels.shape[0], window_size, window_size, input_size), dtype='float32')

# parameter？？
parameter_b = np.array([i - window_size // 2 for i in range(window_size)])

In [5]:
for i in range(features_shape[0]):
    for j in range(features_shape[1]):
        index = i * features_shape[1] + j
        for p in range(window_size):
            for q in range(window_size):
                dataset[index][p][q] = \
                    features[(i + parameter_b[p]) % features_shape[0]][(j + parameter_b[q]) % features_shape[1]]

In [6]:
# 删除所有没有标签的像素
print(labels.shape)
index = np.where(np.reshape(labels, (-1)) == 0)   # 返回坐标tuple(ndarray)
labels = np.delete(labels, index, axis=0)
print(labels.shape)

(21025, 1)
(10249, 1)


In [7]:
tmp = np.unique(labels)   # array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])
for i in range(len(labels)):
    labels[i][0] = np.where(tmp == labels[i][0])[0][0]  # (array([1], dtype=int64),) 第一个[0]索引出元祖中的ndarray，第二[0]索引出ndarray中的int值,处理后，所有的标签值-1 ，因为本来中标签没用0，但是经过Len(labels)，np.where 定位索引位置，又从0开始，所以标签值-1

In [8]:
dataset = np.delete(dataset, index, axis=0)
labels = np.reshape(labels, (-1))  # 降维

In [9]:
dataset = dataset.swapaxes(1, 3)      # 交换轴
print('dataset_shape: ', dataset.shape)


dataset = dataset[:, :, :, :, np.newaxis]     # 添加轴channels
print('dataset_shape: ', dataset.shape)

# before dataset_shape:  (10249, 200, 19, 19)
# after dataset_shape:  (10249, 200, 19, 19, 1)
# print(Y.shape)  # (10249,)

dataset_shape:  (10249, 200, 19, 19)
dataset_shape:  (10249, 200, 19, 19, 1)


In [10]:
# 标准化
dataset = dataset.astype('float32')
dataset -= np.mean(dataset)
dataset /= np.max(dataset)

In [11]:
dataset.shape, labels.shape

((10249, 200, 19, 19, 1), (10249,))

In [12]:
labels = np.reshape(labels, (-1))  # 降维
labels.shape

(10249,)

In [13]:
path_x_all = r"E:\Eric_HSI\excise2\np_data\x_all" + ".npy"
path_y_all = r"E:\Eric_HSI\excise2\np_data\y_all" + ".npy"

np.save(path_x_all, dataset)
np.save(path_y_all, labels)